In [ ]:
import numpy as np
from tensorflow import keras
from keras.layers import (
    Input,
    Add,
    Dense,
    Activation,
    ZeroPadding2D,
    BatchNormalization,
    Flatten,
    Conv2D,
    AveragePooling2D,
    MaxPooling2D,
)
from keras.models import Model
import tensorflow as tf
from tensorflow.python.keras import layers
from sklearn.metrics import precision_recall_fscore_support
import json
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
(X_train, y_train),(X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train = X_train / 255.0
X_test = X_test / 255.0

X_train = X_train.reshape(X_train.shape[0],28,28,1)
X_test = X_test.reshape(X_test.shape[0],28,28,1)

4422102/4422102 [==============================] - 0s 0us/step


In [ ]:
class ResNet18:
  def __init__(self, num_classes = 10, input_shape = (None, None, 3), **kwargs):
    self.num_classes = num_classes
    self.input_shape = input_shape

  def identity_block18(self, x, filter):

    x_skip = x

    x = keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = keras.layers.BatchNormalization(axis=3)(x)
    x = layers.Lambda(lambda x: tf.where(x < 0, tf.tanh(x), x))(x)

    x = keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = keras.layers.BatchNormalization(axis=3)(x)

    x = keras.layers.Add()([x, x_skip])
    x = layers.Lambda(lambda x: tf.where(x < 0, tf.tanh(x), x))(x)
    return x

  def convolutional_block18(self, x, filter):

    x_skip = x

    x = keras.layers.Conv2D(filter, (3,3), padding = 'same', strides = (2,2))(x)
    x = keras.layers.BatchNormalization(axis=3)(x)
    x = layers.Lambda(lambda x: tf.where(x < 0, tf.tanh(x), x))(x)

    x = keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = keras.layers.BatchNormalization(axis=3)(x)

    x_skip = keras.layers.Conv2D(filter, (1,1), strides = (2,2))(x_skip)

    x = keras.layers.Add()([x, x_skip])
    x = layers.Lambda(lambda x: tf.where(x < 0, tf.tanh(x), x))(x)
    return x

  def build(self, **kwargs):

    x_input = Input(self.input_shape)
    x = ZeroPadding2D((3, 3))(x_input)


    x = Conv2D(64, kernel_size=7, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = layers.Lambda(lambda x: tf.where(x < 0, tf.tanh(x), x))(x)
    x = keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)


    name = "ResNet18"
    block_layers = [2, 2, 2, 2]
    filter_size = 64


    for i in range(4):
        if i == 0:
            for j in range(block_layers[i]):
                x = self.identity_block18(x, filter_size)
        else:
            filter_size = filter_size*2
            x = self.convolutional_block18(x, filter_size )
            for j in range(block_layers[i] - 1):
                x = self.identity_block18(x, filter_size)

    x = AveragePooling2D((2,2), padding = 'same')(x)
    x = Flatten()(x)
    x = Dense(self.num_classes, activation = 'softmax')(x)
    model = Model(inputs = x_input, outputs = x, name = name)

    return model

In [ ]:
rn = ResNet18(input_shape=(28, 28, 1))
model = rn.build()
model.summary()

Model: "ResNet18"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 28, 28, 1)]          0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 34, 34, 1)            0         ['input_1[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv2d (Conv2D)             (None, 17, 17, 64)           3200      ['zero_padding2d[0][0]']      
                                                                                                  
 batch_normalization (Batch  (None, 17, 17, 64)           256       ['conv2d[0][0]']       

In [ ]:
model.compile(
    optimizer="Adam",

    loss="sparse_categorical_crossentropy",

    metrics=["sparse_categorical_accuracy"],
)

learning = model.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs=30, verbose=1
)

loss_train = learning.history["loss"]

loss_val = learning.history["val_loss"]

acc_train = learning.history["sparse_categorical_accuracy"]

acc_val = learning.history["val_sparse_categorical_accuracy"]

y_pred = [np.argmax(arr) for arr in model.predict(X_test)]

prf_score = precision_recall_fscore_support(y_test, y_pred, average="weighted")

score = model.evaluate(X_test, y_test)

Epoch 1/30
1875/1875 [==============================] - 62s 25ms/step - loss: 0.4290 - sparse_categorical_accuracy: 0.8455 - val_loss: 0.3925 - val_sparse_categorical_accuracy: 0.8583
Epoch 2/30
1875/1875 [==============================] - 44s 24ms/step - loss: 0.2996 - sparse_categorical_accuracy: 0.8890 - val_loss: 0.2979 - val_sparse_categorical_accuracy: 0.8904
Epoch 3/30
1875/1875 [==============================] - 44s 24ms/step - loss: 0.2659 - sparse_categorical_accuracy: 0.9018 - val_loss: 0.3021 - val_sparse_categorical_accuracy: 0.8940
Epoch 4/30
1875/1875 [==============================] - 44s 23ms/step - loss: 0.2428 - sparse_categorical_accuracy: 0.9101 - val_loss: 0.2800 - val_sparse_categorical_accuracy: 0.8960
Epoch 5/30
1875/1875 [==============================] - 43s 23ms/step - loss: 0.2207 - sparse_categorical_accuracy: 0.9165 - val_loss: 0.2592 - val_sparse_categorical_accuracy: 0.9059
Epoch 6/30
1875/1875 [==============================] - 44s 24ms/step - loss: 0.

In [ ]:
performance = {
    'loss_train': loss_train,
    'loss_val': loss_val,
    'acc_train': acc_train,
    'acc_val': acc_val,
    'prf_score': prf_score,
    'score': score
}

performance_file_path = '/content/drive/My Drive/Colab Notebooks/custom1_on_fmnist_performance.json'

with open(performance_file_path, 'w') as f:
    json.dump(performance, f)

print("performance saved to:", performance_file_path)

performance saved to: /content/drive/My Drive/Colab Notebooks/custom1_on_fmnist_performance.json
